## LER Classifier Notebook

This notebook extracts LERs, downloaded from ADAMs, fetch relevant IRs and creates a classifier to identify IRs that are promoted to LER vs others. This will then find, based on probability, IRs that are borderline cases. 

Note: This notebook must be run outside dev container to run successfully. Pyodbc throws error when run inside container.

To run outside, you need this notebook and dependent libraries (alcs_global.py, alcs_llm.py) and fix relative path below
to reference them correctly.

### Narrative
While creating this final version of the code, various experiments were done to reach at final set of features. 
File "./log/experiment_log.txt" captures those experiment results with various features. Please read the file for the details.

Also, this code generates pred_full.csv file that contains the probability of full dataset and various IRs to be LERs. Below code find the best threshold for separating positive and negative classes. Though LERs set is statis, as it comes from NRC, Non-LERs will change
with every run.

In [ ]:
# %pip install -r requirements.txt

In [ ]:
# Environment variable keys
env_var_openai_key = 'NUREG_AZURE_OPENAI_SERVICE_KEY'
env_var_openai_uri = 'NUREG_AZURE_OPENAI_SERVICE_URI'
env_var_openai_model = 'NUREG_AZURE_OPENAI_CHATGPT_MODEL'
env_var_openai_embedding_model = 'SEARCH_EVAL_OPENAI_EMBEDDING_MODEL'
env_var_storage_container_name = "AZURE_STORAGE_CONTAINER_NAME"
env_var_storage_connection_string = "AZURE_STORAGE_CONNECTION_STRING"

# Add custom library path
import sys
sys.path.append('../../src/evaluation/search/helper')


In [ ]:
# Import Libraries
import pyodbc, os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from urllib.parse import quote
from alcs_llm import AzureOpenAIModel, AzureOpenAIService
from alcs_global import Common
from azure.storage.blob import BlobServiceClient
import shutil
from pathlib import Path
import json
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import seaborn as sns
from scipy.spatial.distance import pdist, squareform


### Initializing

In [ ]:
# Load environment variables
load_dotenv()

# Set paramters for Azure OpenAI
azure_openai_model = AzureOpenAIModel(
    open_api_key = os.getenv(env_var_openai_key),
    open_api_uri = os.getenv(env_var_openai_uri),
    chatgpt_model_id = os.getenv(env_var_openai_model)
)

# Initialize Azure OpenAI
azure_openai = AzureOpenAIService(azure_openai_model = azure_openai_model)

# Set embedding model from env
embedding_model=os.getenv(env_var_openai_embedding_model)

container_name = os.getenv(env_var_storage_container_name)
connection_string = os.getenv(env_var_storage_connection_string)
local_download_path = "output/lers"
blob_folder_name = "processed"
blob_file_filter = ".txt"
ler_output_csv = "output/lers/ler_catalog.csv"
ler_ir_map_csv = "log/ler_ir_map.csv"
test_pred_csv = "log/pred_test.csv"
full_pred_csv = "log/pred_full.csv"

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

### Download LERs summary text files.

In [ ]:
# Ensure the local directory exists
if os.path.exists(local_download_path):
    shutil.rmtree(local_download_path)
os.makedirs(local_download_path, exist_ok=True)

# List and download all blobs in the container
print("Downloading blobs...")
blob_urls = []
for blob in container_client.list_blobs(name_starts_with=blob_folder_name): 

    if blob.name.endswith(blob_file_filter):
        blob_name = blob.name
        local_file_path = os.path.join(local_download_path, blob_name)

        blob_url = f"{container_client.url}/{blob.name}"
        blob_urls.append(blob_url)

        # Create directories for nested blobs
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

        # Download the blob
        with open(local_file_path, "wb") as file:
            blob_client = container_client.get_blob_client(blob_name)
            file.write(blob_client.download_blob().readall())
        # print(f"Downloaded: {blob_name}")

print(f"{len(blob_urls)} blobs downloaded successfully!")


### Consolidate LER meta info into single file

In [ ]:
rows = []
data_dir = Path(f"{local_download_path}/{blob_folder_name}")
for file in data_dir.glob("*.txt"):
    with open(file, "r") as f:
        try:
            records = json.load(f)
            rows.append({
                "EVENTDATE": records.get("event_date", ""),
                "REPORTDATE": records.get("report_date", ""),
                "LERNUMBER": records.get("ler_number", ""),
                "FACILITYNAME": records.get("facility_name", ""),
                "TITLE": records.get("title", ""),
                # "content": f"Abstract:\n{records.get('abstract', '')}\n\n{records.get('narrative', '')}",
                "SUBSECTIONS": ", ".join([f"10 CFR {subsection}" for subsection in records.get("cfr_requirements",[])]),
                "FILENAME": os.path.basename(f.name)
                })
        except json.JSONDecodeError:
            print(f"❌ Failed to parse {file}")

if rows:
    df = pd.DataFrame(rows)
    df.to_csv(ler_output_csv, index=False)
    print(f"✅ Generated ground truth CSV with {len(rows)} entries at {ler_output_csv}")
else:
    print("⚠️ No valid LERs found to include in ground truth.")

In [ ]:
# Read LER CSV
df = pd.read_csv(ler_output_csv)

### Create Constellations Facility Dataset

In [ ]:
# List of Constellations Nuclear Facility Code and Name
constellation_nuclear_facility = [{"code":"CAL", "name":"Calvert Cliffs", "alt":""},
                                  {"code":"GIN", "name":"Ginna", "alt":"GNA"},
                                  {"code":"DRE", "name":"Dresden", "alt":""},
                                  {"code":"QDC", "name":"Quad Cities", "alt":""},
                                  {"code":"JAF", "name":"Fitzpatrick", "alt":""},
                                  {"code":"CPS", "name":"Clinton", "alt":""},
                                  {"code":"NMP", "name":"Nine Mile Point", "alt":""},
                                  {"code":"LAS", "name":"LaSalle", "alt":""},
                                  {"code":"LIM", "name":"Limerick", "alt":"LGS"},
                                  {"code":"PEA", "name":"Peach Bottom", "alt":""},
                                  {"code":"TMI", "name":"Three Mile Island", "alt":""},
                                  {"code":"BYR", "name":"Byron", "alt":""},
                                  {"code":"BRW", "name":"Braidwood", "alt":""},
                                  {"code":"ZIN", "name":"Zion", "alt":""},
                                  {"code":"OYS", "name":"Oyster Creek", "alt":""},
                                  {"code":"NCS", "name":"Nuclear Corporate", "alt":""}]

# Create a mapping dictionary from the list of dictionaries
mapping_dict = {item['name']: item['code'] for item in constellation_nuclear_facility}

# Accessing the dictionary
for item in constellation_nuclear_facility:
    print(f"Code: {item['code']}, Name: {item['name']}, AlternateCode: {item['alt']}")

### Functions

In [ ]:
# Function to check if the dictionary contains the target value
def search_dict(text, nuclear_facility_list):
    pattern = r'\b(?:' + '|'.join(map(re.escape, nuclear_facility_list)) + r')\b'
    return bool(re.search(pattern, text))

In [ ]:
# Function to assign codes based on patterns
def assign_code(text):
    for facility, code in mapping_dict.items():
        if facility in text:
            return code
    return 'Unknown' 

In [ ]:
# Function to calculate cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

### Clean data

In [ ]:
# pd.set_option('display.max_rows', None)
df_cleaned = df.dropna(subset=['FACILITYNAME'])
df_cleaned = df_cleaned[~df_cleaned['EVENTDATE'].str.contains('None', case=False, na=False)]
df_cleaned = df_cleaned[~df_cleaned['EVENTDATE'].str.contains('\n', case=False, na=False)]
df_cleaned = df_cleaned[~df_cleaned['EVENTDATE'].str.contains(r'\.', case=False, na=False)]

df_cleaned['EVENT_DATE'] = pd.to_datetime(df_cleaned['EVENTDATE'], format='mixed').dt.strftime('%Y%m%d')

In [ ]:
# Apply the function to the DataFrame
facility_names = [item["name"] for item in constellation_nuclear_facility]
df_cleaned['CONTAINS_FACILITY'] = df_cleaned['FACILITYNAME'].apply(lambda x: search_dict(x, facility_names))
df_cleaned['FACILITY_CODE'] = df_cleaned['FACILITYNAME'].apply(assign_code)
df_constellation_lers = df_cleaned[df_cleaned['FACILITY_CODE']!='Unknown']
df_constellation_lers = df_constellation_lers[~df_constellation_lers[['EVENT_DATE', 'FACILITY_CODE']].duplicated()]
print(len(df_constellation_lers))

In [ ]:
# Facility to LER distribution
sns.histplot(data=df_constellation_lers, x="FACILITY_CODE")

In [ ]:
# Function for fetching classifier dataset from AS9
def fetch_classifier_dataset(df):
    conn_str = (
        f"DRIVER={{Oracle in OraClient19Home1}};"
        f"DBQ=AS9NUCRP;"
        f"UID=;"
        f"PWD=;"
    )
 
    eventDates = tuple(df['EVENT_DATE'].unique())
    facilities = tuple(df['FACILITY_CODE'].unique())

    query = f"""
                WITH
                -- POSITIVE CLASS CTE 
                LER_POSITIVE_CTE AS (
 
                    SELECT 
                        DISTINCT (CASE WHEN T1.AR_NUMBER = '04501492' THEN '04501564'
                                    ELSE T1.AR_NUMBER
                                END) AS AR_NUMBER,
                        T1.AR_ORIG_FACILITY,
                        T1.EVENT_DATE,
                        T1.AR_PRIORITY,
                        T1.AR_SEVERITY
                    FROM TIDARMST T1
                    JOIN TIDASMST T2
                        ON T1.AR_NUMBER = T2.AR_NUMBER
                    WHERE 
                    T2.ASSIGN_TYPE = 'LER'
                    AND TRIM(T1.AR_ORIG_FACILITY) IS NOT NULL
                    AND TRIM(T1.EVENT_DATE) IS NOT NULL
                    AND T1.AR_TYPE = 'CR'
                    AND T1.EVENT_DATE IN {eventDates}
                    AND T1.AR_ORIG_FACILITY IN {facilities}
                ),
            
                -- NEGATIVE CLASS CTE
                LER_NEGATIVE_CTE AS (
                    SELECT * 
                    FROM (   
                        SELECT 
                            DISTINCT T1.AR_NUMBER,
                            T1.AR_ORIG_FACILITY,
                            T1.EVENT_DATE,
                            T1.AR_PRIORITY,
                            T1.AR_SEVERITY
                        FROM TIDARMST T1
                        WHERE T1.AR_NUMBER NOT IN (SELECT AR_NUMBER FROM LER_POSITIVE_CTE)
                            AND TRIM(T1.AR_ORIG_FACILITY) IS NOT NULL
                            AND TRIM(T1.EVENT_DATE) IS NOT NULL
                            AND T1.AR_TYPE = 'CR'
                            AND T1.EVENT_DATE >= (SELECT MIN(EVENT_DATE) FROM LER_POSITIVE_CTE)
                            AND T1.AR_SUBJECT NOT LIKE '%AUTO-CANCELED%'
                            AND TRIM(T1.AR_SEVERITY) IS NOT NULL
                            AND TRIM(T1.AR_PRIORITY) IS NOT NULL
                        ORDER BY DBMS_RANDOM.VALUE
                        )
                    WHERE ROWNUM <= 565
                )

                -- POSITIVE CLASS
                SELECT T1.AR_NUMBER,
                        T1.AR_ORIG_FACILITY,
                        T1.EVENT_DATE,
                        T1.AR_PRIORITY,
                        T1.AR_SEVERITY,
                        RTRIM(
                            REPLACE(
                                REPLACE(
                                    XMLAGG(
                                        XMLELEMENT("x", REGEXP_REPLACE(T2.DESCRIPTION_NOTES, '[^[:print:]]', '') || ' ')
                                        ORDER BY T2.GEN_ARG
                                    ).GetClobVal(),
                                    '<x>', ''),
                                '</x>', '')
                            , ' ') AS CONTENT,
                        1 AS LER_LABEL
                FROM LER_POSITIVE_CTE T1
                JOIN TIDARCOM T2
                    ON T1.AR_NUMBER = T2.AR_NUMBER
                WHERE T2.AR_COMMENT_TYPE = 'D'
                GROUP BY T1.AR_NUMBER, T1.AR_ORIG_FACILITY, T1.EVENT_DATE, T1.AR_PRIORITY, T1.AR_SEVERITY

                UNION ALL
                
                -- NEGATIVE CLASS
                SELECT T1.AR_NUMBER,
                        T1.AR_ORIG_FACILITY,
                        T1.EVENT_DATE,
                        T1.AR_PRIORITY,
                        T1.AR_SEVERITY,
                        RTRIM(
                            REPLACE(
                                REPLACE(
                                    XMLAGG(
                                        XMLELEMENT("x", REGEXP_REPLACE(T2.DESCRIPTION_NOTES, '[^[:print:]]', '') || ' ')
                                        ORDER BY T2.GEN_ARG
                                    ).GetClobVal(),
                                    '<x>', ''),
                                '</x>', '')
                            , ' ') AS CONTENT,
                        0 AS LER_LABEL
                FROM LER_NEGATIVE_CTE T1
                JOIN TIDARCOM T2
                    ON T1.AR_NUMBER = T2.AR_NUMBER
                WHERE T2.AR_COMMENT_TYPE = 'D'
                GROUP BY T1.AR_NUMBER, T1.AR_ORIG_FACILITY, T1.EVENT_DATE, T1.AR_PRIORITY, T1.AR_SEVERITY
    """
 
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql(query, conn)
    conn.close()

    df = df[df['CONTENT'].str.strip().astype(bool)]

    return df
 

In [ ]:
# Get dataset
df = fetch_classifier_dataset(df_constellation_lers)
print(f"Fetched {len(df)} records")

### Save LER to IR mapping

In [ ]:
columns = ["AR_NUMBER", "LERNUMBER", "EVENT_DATE", "AR_ORIG_FACILITY", "LER_LABEL", "FILENAME"]

df_lers = pd.merge(df[df["LER_LABEL"]==1], df_constellation_lers, how='left', 
                            left_on=['EVENT_DATE', 'AR_ORIG_FACILITY'], right_on=['EVENT_DATE', 'FACILITY_CODE'])
df_lers = df_lers[columns].to_csv(ler_ir_map_csv, index=False)

### Extract document sections and create embeddings

In [ ]:
# Regex pattern to match and extract
pattern1 = r"Condition Description:\s*(.*?)(?=\s*(Immediate actions taken|Recommended Actions):|$)"
pattern2 = r"Immediate actions taken:\s*(.*?)(?=\s*(Recommended Actions|Initial Screening Questions):|$)"

# Clean dataset
df['COND_DESC'] = df['CONTENT'].apply(lambda text: Common.preprocess_text(text, pattern1, clean_text=False))
df['IMME_ACTN'] = df['CONTENT'].apply(lambda text: Common.preprocess_text(text, pattern2, clean_text=False))


# Generate embeddings using default vectorization (ada-002)
X_COND_DESC = azure_openai.generate_embeddings(entries=df['COND_DESC'],
                                            embedding_model_id=embedding_model)
if len(X_COND_DESC) == 0:
    print('\t\tError generating default embeddings')
    sys.exit("Errors!")

X_IMME_ACTN = azure_openai.generate_embeddings(entries=df['IMME_ACTN'],
                                            embedding_model_id=embedding_model)
if len(X_IMME_ACTN) == 0:
    print('\t\tError generating default embeddings')
    sys.exit("Errors!")

df['VEC_DEF_1'] = X_COND_DESC
df['VEC_DEF_2'] = X_IMME_ACTN

df = df[df['VEC_DEF_1'] != -99999.99999]
df = df[df['VEC_DEF_2'] != -99999.99999]

X_full = pd.concat([df['VEC_DEF_1'].apply(pd.Series), df['VEC_DEF_2'].apply(pd.Series)], axis=1)
X_full['FACILITY'] = df['AR_ORIG_FACILITY'].astype('category')
y_full = df['LER_LABEL'].astype(int)

### Fixed Train-Test Split

In [ ]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=43)

### Separate Categorical and Non-Categorical features

In [ ]:
X_train_noncat = X_train.iloc[:,:-1]
X_test_noncat = X_test.iloc[:,:-1]
X_train_cat = X_train[['FACILITY']]
X_test_cat = X_test[['FACILITY']]

### Scale and Reduce Dimensions

In [ ]:
# Scale features before PCA
scaler = StandardScaler()
scaler.fit(X_train_noncat)

X_train_scaled = scaler.transform(X_train_noncat)
X_test_scaled = scaler.transform(X_test_noncat)

# Fit PCA
pca = PCA()
pca.fit(X_train_scaled)

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Find the number of components that explain at least 95% variance
n_components = np.argmax(cumulative_variance >= 0.95) + 1

print(f"Number of components explaining 95% variance: {n_components}")

# Apply PCA with the selected number of components
pca_95 = PCA(n_components=n_components)
pca_95.fit(X_train_scaled)

X_train_pca = pd.DataFrame(pca_95.transform(X_train_scaled))
X_test_pca = pd.DataFrame(pca_95.transform(X_test_scaled))


### One-Hot Encoding of Categorical Features

In [ ]:
# one-hot encode the categorical features
cat_attribs = ['FACILITY']

transformer = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)], remainder='passthrough')

encoder = transformer.fit(X_train_cat)
X_train_encode =  pd.DataFrame(encoder.transform(X_train_cat).toarray())
X_test_encode = pd.DataFrame(encoder.transform(X_test_cat).toarray())

### Combine both above dataset to create final feature dataset

In [ ]:
# Combine both datasets
X_train_final = pd.concat([X_train_pca.reset_index(), X_train_encode.reset_index()], axis=1, ignore_index=True)
X_test_final = pd.concat([X_test_pca.reset_index(), X_test_encode.reset_index()], axis=1, ignore_index=True)

# Rename columns
X_train_final.columns = [f'col_{i}' for i in range(X_train_final.shape[1])]
X_test_final.columns = [f'col_{i}' for i in range(X_test_final.shape[1])]

print(X_train_final.shape)
print(X_test_final.shape)

### Train model and Predict

In [ ]:
# Initialize the XGBoost classifier
xgb_model = XGBClassifier(n_estimators=10, max_depth=4, learning_rate=0.05)

# Train the model
xgb_model.fit(X_train_final, y_train)

# Predict probabilities
y_test_proba = xgb_model.predict_proba(X_test_final)[:, 1]
y_train_proba = xgb_model.predict_proba(X_train_final)[:, 1]


In [ ]:
# Plot Precision-Recall with threshold to analyse visually

precision, recall, thresholds = precision_recall_curve(y_test, y_test_proba)
auc_score = auc(recall, precision)

plt.plot(thresholds, precision[:-1], label="Precision")
plt.plot(thresholds, recall[:-1], label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.legend()
plt.title("Precision-Recall vs Threshold")

In [ ]:
# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall Curve (AUC = {auc_score:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

In [ ]:
# Find best threshold
f1_scores = 2 * recall * precision / (recall + precision)
best_thresh = thresholds[np.argmax(f1_scores)]
best_thresh

In [ ]:
y_test_pred = (y_test_proba > best_thresh).astype(int)

out_df = pd.DataFrame({"Pred": y_test_pred, "True": y_test, "Prob":y_test_proba})
out_df = out_df.sort_values(by='Prob')
out_df.to_csv(test_pred_csv, index=False)

### Run full data thru the model

In [ ]:
X_full_noncat = X_full.iloc[:,:-1]
X_full_cat = X_full[['FACILITY']]

X_full_scaled = scaler.transform(X_full_noncat)
X_full_pca = pd.DataFrame(pca_95.transform(X_full_scaled))
X_full_encode =  pd.DataFrame(encoder.transform(X_full_cat).toarray())
X_full_final = pd.concat([X_full_pca.reset_index(), X_full_encode.reset_index()], axis=1, ignore_index=True)

X_full_final.columns = [f'col_{i}' for i in range(X_full_final.shape[1])]

y_pred_proba_full = xgb_model.predict_proba(X_full_final)[:, 1]
y_pred_full = (y_pred_proba_full > best_thresh).astype(int)


print("--------------- TRAIN SCORES ------------------")
print(f"Accuracy: {accuracy_score(y_full, y_pred_full):.2f}")
print(f"Precision: {precision_score(y_full, y_pred_full)}")
print(f"Recall: {recall_score(y_full, y_pred_full)}")
print(f"F1 Score: {f1_score(y_full, y_pred_full):.2f}")
# Confusion Matrix
cfm = confusion_matrix(y_full, y_pred_full).flatten()
print(f"Confusion Matrix: TP -> {cfm[3]}, FN -> {cfm[2]}, FP -> {cfm[1]}, TN -> {cfm[0]}")


### Save full dataset output

In [ ]:
out_df = pd.DataFrame({"Pred": y_pred_full, 
                       "True": y_full, 
                       "Prob": y_pred_proba_full})
df_complete = pd.concat([df, out_df], axis=1)
df_complete[["AR_NUMBER", "Pred","True","Prob"]].sort_values(by='Prob').to_csv(full_pred_csv, index=False)

In [ ]:
# Create the histogram
sns.histplot(data=out_df, x="Prob", hue="Pred", kde=True)

In [ ]:
distance_matrix_full = squareform(pdist(df_complete["VEC_DEF_1"].apply(pd.Series), metric='cosine'))
distance_df_full = pd.DataFrame(distance_matrix_full, index=df_complete.index, columns=df_complete.index)
# distance_df_full

In [ ]:
# Plot heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(distance_df_full, annot=False, cmap='coolwarm', fmt=".2f")
plt.title("Row-wise Distance Heatmap")
plt.show()

### Filter records for analysis

In [ ]:
df_analysis = df_complete[(df_complete["Prob"]>=(best_thresh - 0.05)) & 
                          (df_complete["Prob"]<=(best_thresh + 0.05)) & 
                          (df_complete["Pred"]!=df_complete["True"])]
len(df_analysis)

In [ ]:
distance_matrix = squareform(pdist(df_analysis["VEC_DEF_1"].apply(pd.Series), metric='cosine'))
distance_df = pd.DataFrame(distance_matrix, index=df_analysis.index, columns=df_analysis.index)

In [ ]:
# Mismatched records
sns.histplot(data=df_analysis, x="Prob", hue="True", kde=True)


In [ ]:
# Plot heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(distance_df, annot=False, cmap='coolwarm', fmt=".2f")
plt.title("Row-wise Distance Heatmap")
plt.show()

In [ ]:
df_analysis[(df_analysis["True"] == 1) & (df_analysis["Pred"] == 0)].head(5)

In [ ]:
df_analysis[(df_analysis["True"] == 0) & (df_analysis["Pred"] == 1)].head(5)

In [ ]:
import textwrap
pd.set_option('display.max_colwidth', None)

In [ ]:
# LER
wrapped_column= df_analysis[df_analysis["AR_NUMBER"]=='01299156']['COND_DESC'].str.wrap(width=40)
for value in wrapped_column:
    print(value)


In [ ]:
# NON LER
wrapped_column= df_analysis[df_analysis["AR_NUMBER"]=='02449287']['COND_DESC'].str.wrap(width=40)
for value in wrapped_column:
    print(value)